In [1]:
import pymc3

/home/oskarsinger/anaconda3/lib/python3.5/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')
/home/oskarsinger/anaconda3/lib/python3.5/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [2]:
import numpy as np

In [3]:
T = 1000
k = 5
d1 = 20
d2 = 30

In [4]:
W1 = np.random.randn(d1, k)
W2 = np.random.randn(d2, k)
mu1 = np.random.randn(d1, 1)
mu2 = np.random.randn(d2, 1)
pre_Psi1 = np.random.randn(2 * d1, d1)
Psi1 = np.dot(pre_Psi1.T, pre_Psi1)
pre_Psi2 = np.random.randn(2 * d2, d2)
Psi2 = np.dot(pre_Psi2.T, pre_Psi2)

In [7]:
Psi1_sqrt = np.linalg.cholesky(Psi1)
Psi2_sqrt = np.linalg.cholesky(Psi2)

In [9]:
Z = np.random.randn(T, k)
Z_lift1 = np.dot(W1, Z.T).T
Z_lift2 = np.dot(W2, Z.T).T
Z_shift1 = Z_lift1 + mu1.T
Z_shift2 = Z_lift2 + mu2.T
X1_noise = np.random.randn(T, d1)
X2_noise = np.random.randn(T, d2)
X1 = np.dot(X1_noise, Psi1_sqrt) + Z_shift1
X2 = np.dot(X2_noise, Psi2_sqrt) + Z_shift2

In [10]:
import pymc3 as pm

In [15]:
model = pm.Model()

with model:
    # Priors on Z-lifting matrices
    W1_var = pm.Normal('W1', shape=(d1,k))
    W2_var = pm.Normal('W2', shape=(d2,k))
    
    # Priors on X means
    mu1_var = pm.Normal('mu1', shape=(d1,1))
    mu2_var = pm.Normal('mu2', shape=(d2,1))
    
    # Priors on covariance matrices
    Psi1_var = pm.LKJCholeskyCov(
        'Psi1', n=d1, eta=2., sd_dist=pm.HalfCauchy.dist(2.5))
    Psi2_var = pm.LKJCholeskyCov(
        'Psi1', n=d2, eta=2., sd_dist=pm.HalfCauchy.dist(2.5))
    
    # Likelihood on Z
    Z_var = pm.Normal('Z', shape=(T,k))
    
    # Means for conditional likelihood on Xs conditioned on Z
    Z_lift1_var = np.dot(W1_var, Z_var.T).T
    Z_lift2_var = np.dot(W2_var, Z_var.T).T
    Z_shift1_var = Z_lift1_var + mu1_var.T
    Z_shift2_var = Z_lift2_var + mu2_var.T
    
    # Conditional likelihoods of Xs conditioned on Z
    X1_var = pm.Normal('X1', mean=Z_shift1_var, sd=SD1_var, observed=X1)
    X2_var = pm.Normal('X2', mean=Z_shift2_var, sd=SD2_var, observed=X2)

LinAlgError: 0-dimensional array given. Array must be at least two-dimensional